# Displaying prediction with real optical images

In [4]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba

Storing the required variables(R,G,B) related to each latitude and longitude in a single list 

In [2]:
import pandas as pd
import pystac_client

# Load the CSV file containing combined latitude and longitude values
df = pd.read_csv("Our prediction.csv")

# Define the time window
time_window = "2022-12-31"

# Initialize an empty list to store the results for each lat-long pair
result_list = []
 # Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 20  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract latitude and longitude values from the 'Lat_Long' column
    lat_long_str = row['Latitude and Longitude']
    lat_long_str = lat_long_str.strip('()')  # Remove parentheses
    lat_str, lon_str = lat_long_str.split(', ')  # Split into latitude and longitude strings
    lat = float(lat_str)
    lon = float(lon_str)

    # Calculate the Lat-Lon bounding box region
    box_size_deg = 0.04  # Surrounding box in degrees
    min_lon = lon - box_size_deg / 2
    min_lat = lat - box_size_deg / 2
    max_lon = lon + box_size_deg / 2
    max_lat = lat + box_size_deg / 2
    bounds = (min_lon, min_lat, max_lon, max_lat)

    # Open the STAC client and search for items
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items())
    #print('This is the number of scenes that touch our region:',len(items))
   
    if items:
        # Load data for the current lat-long pair
        xx = stac_load(
            items,
            bands=["red", "green", "blue"],
            crs="EPSG:4326",  # Latitude-Longitude
            resolution=scale,  # Degrees
            chunks={"x": 2048, "y": 2048},
            dtype="uint16",
            patch_url=pc.sign,
            bbox=bounds
        )

        # Append the result to the list
        result_list.append(xx)
    else:
        print(f"No items found for Lat: {lat}, Lon: {lon}")

# Now, result_list contains the data for each lat-long pair
# You can process this list further as needed


In [3]:
Plotting the rgb of each Latitude and longitude

SyntaxError: invalid syntax (1134067396.py, line 1)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have another DataFrame 'other_df' with a column 'Title_Column'
# Iterate through each data array in result_list (assuming result_list contains all xx arrays)
for index, data_array in enumerate(result_list):
    # Extract latitude and longitude values from the data_array
    lat = data_array.coords['latitude'].values[0]
    lon = data_array.coords['longitude'].values[0]

    # Create a plot for the red, green, and blue bands
    plot_data = data_array[["red", "green", "blue"]].to_array()

    # Get the title from 'other_df' based on the index
    if index < len(df):
        existing_title = df.loc[index, 'Prediction']
    else:
        existing_title = f"Title not available for index {index}"

    # Modify the title to include index and "Predicted"
    title = f" {index}.Predicted: {existing_title}"

    # Plot the data
    plot_data.plot.imshow(col='time', col_wrap=1, robust=True, vmin=0, vmax=3000)

    # Set the title for the plot
    plt.title(title)

    # Add text annotations for latitude and longitude on the plot
    plt.text(0.02, 0.90, f"Lat: {lat:.2f}", transform=plt.gca().transAxes, fontsize=10, color='white')
    plt.text(0.02, 0.80, f"Long: {lon:.2f}", transform=plt.gca().transAxes, fontsize=10, color='white')

    # Remove latitude and longitude ticks on both the x and y axes
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    
    # Remove x and y axis labels (titles)
    plt.gca().set_xlabel('')
    plt.gca().set_ylabel('')
    
    # Show the plot
    plt.show()
